In [47]:
from xml.dom.minidom import Document

# 导入模型
from langchain_community.llms import Ollama
llm = Ollama(model="qwen2.5:7b")

In [48]:
# 普通提问
llm.invoke("你知道矿小助吗")

'“矿小助”这个名字听起来像是一个提供矿业相关服务或技术支持的虚拟助手的名字。不过，具体到某个特定的服务或者产品，“矿小助”目前没有明确广泛认知的信息。有可能是某个公司内部开发的应用程序、服务平台或者是社交媒体账号的一部分名称。\n\n如果你是在询问某款软件、工具或是某个公司的客户服务代表的话，请提供更多细节信息，我会尽力提供帮助或解答你的疑问。'

In [49]:
# 使用 prompt | llm 链
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手"),
    ("user", "{input}")
])
chain = prompt | llm
temp = chain.invoke({"input": "你是谁"})
print(temp)

我是Qwen，一个由阿里云开发的AI助手。我可以帮助解答问题、提供信息和进行各种对话。您有什么想要了解或讨论的话题吗？


In [50]:
# 使用 prompt | llm | output_parser 链
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "你好"})

'你好！很高兴遇见你。有什么问题或者需要帮助的吗？'

In [51]:
# 设置代码中的USER_AGENT环境
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

In [52]:
# 通过bs4引入外部数据
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.cumt.edu.cn/info/1128/67688.htm")

docs = loader.load()

In [53]:
# 引入embedding model
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="qwen2.5:7b")

In [54]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter() # 递归字符分割器
documents = text_splitter.split_documents(docs) # 分割文档
print(documents)
vector = FAISS.from_documents(documents, embeddings) # EMBED向量化
# vector.save_local()
print(vector.embeddings)


[Document(metadata={'source': 'https://www.cumt.edu.cn/info/1128/67688.htm', 'title': '关于2024年中秋节、国庆节放假安排的通知-中国矿业大学', 'description': '各学院、部、处、室，各有关单位：根据《国务院办公厅关于2024年部分节假日安排的通知》（国办发明电〔2023〕7号），结合学校实际，现将我校2024年中秋节、国庆节放假安排通知如下：一、时间安排1.中秋节：9月15日（...', 'language': 'en'}, page_content='关于2024年中秋节、国庆节放假安排的通知-中国矿业大学\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n在校学生\n\n\n\n在校学生\n教职员工\n毕业校友\n未来学生\n\n\n\n\n\nEN\n\n\n\n\n \n\n\n\n\n\n\n\n\n \n\n 网站首页\n\n 学校概况\n\n\n学校简介\n\n矿大章程\n\n历史沿革\n\n历任领导\n\n现任领导\n\n基本数据\n\n知名学者\n\n校园文化\n\n虚拟校园\n\n联系我们\n\n\n\n\n 机构设置\n\n\n党群组织\n\n行政管理\n\n学院设置\n\n科研机构\n\n直附属单位\n\n产业单位\n\n其他\n\n\n\n\n 教育教学\n\n\n教务部\n\n研究生院\n\n国际学院\n\n继续教育学院\n\n教师教学发展中心\n\n在线学习（MOOC）平台\n\n图书馆\n\n公共教学服务中心\n\n\n\n\n 科学研究\n\n\n科学技术研究院\n\n人文社会科学处\n\n科研平台\n\n科研项目\n\n科研团队\n\n科研成果\n\n学术期刊\n\n现代分析与计算中心\n\n\n\n\n\n\n\n\n\n \n \n\n 学科建设\n\n\n学科建设处\n\n“双一流”建设\n\n重点学科\n\n学位授权点\n\n\n\n\n 合作交流\n\n\n国际合作交流处\n\n国际矿业联盟\n\n对外合作与发展处\n\n理事会\n\n校友会\n\n教育发展基金会\n\n矿大培训网\n\n\n\n\n 人才招聘\n\n\n人力资源部\

In [55]:
# 创建文档检索链

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""根据提供的上下文回答以下问题:

<context>
{context}, 
据可靠消息知，矿大2023年国庆放10天假
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [56]:
# 从文档中检索

document_chain.invoke({
    "input": "矿大2023和2024年国庆分别放多少天假",
    "context": documents
})

'根据提供的信息：\n\n- 2024年国庆节放假安排为10月1日至7日共7天，9月29日（星期日）和10月12日（星期六）上班。\n\n关于2023年的国庆放假情况，您提到“据可靠消息知，矿大2023年国庆放10天假”，但提供的上下文中并没有直接提及2023年的具体放假安排。因此，可以初步确认的是，2024年国庆节放假7天，而2023年的国庆放假情况则为10天。\n\n如果有进一步的官方文件或通知能明确2023年的具体假期天数，建议参考该信息以获得准确答案。'

In [57]:
# 使用检索器检索

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "矿大2023和2024年国庆分别放多少天假?"})
print(response["answer"])

根据提供的信息:

- 对于2024年的国庆节放假安排,通知中明确指出放假时间为10月1日至7日,共7天。但同时提到9月29日(星期日)和10月12日(星期六)上班。

- 关于2023年国庆假期的具体时长，信息中仅提及“据可靠消息知，矿大2023年国庆放10天假”，但是没有提供具体放假日期或进一步的官方通知作为依据。因此，对于2023年的国庆节放假安排时长为10天,但具体安排未详细说明。

综上所述:
- 2024年国庆节放假7天。
- 2023年国庆节放假10天（根据消息）。
